# Prácticas Tema : Compresión. Huffman, truncado de bloques y Transformada de Haar

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

### Ejercicio 1: codificación Huffman

A continuación tienes un código (obtenido de internet tras una búsqueda muy corta) en el que, a partir de un diccionario de valores junto con su frecuencia, nos devuelve un diccionario siguiendo la codificación Huffman. 

In [2]:
def huffman(freqtable):
    from collections import defaultdict 
    from heapq import heappush, heappop, heapify

    # mapping of letters to codes
    code = defaultdict(list)

    # Using a heap makes it easy to pull items with lowest frequency.
    # Items in the heap are tuples containing a list of letters and the
    # combined frequencies of the letters in the list.
    heap = [ ( freq, [ ltr ] ) for ltr,freq in freqtable.items() ]
    heapify(heap)

    # Reduce the heap to a single item by combining the two items
    # with the lowest frequencies.
    while len(heap) > 1:
        freq0,letters0 = heappop(heap)
        for ltr in letters0:
            code[ltr].insert(0,'0')

        freq1,letters1 = heappop(heap)
        for ltr in letters1:
            code[ltr].insert(0,'1')

        heappush(heap, ( freq0+freq1, letters0+letters1))

    for k,v in code.items():
        code[k] = ''.join(v)

    return code

freqtable = dict(a=45, b=13, c=12, d=16, e=9, f=5)
print(sorted(huffman(freqtable).items()))

[('a', '0'), ('b', '101'), ('c', '100'), ('d', '111'), ('e', '1101'), ('f', '1100')]


A continuación, diseña un código que tome como parámetro el nombre de una imagen que tengas dentro de la carpeta images asociada a esta práctica (la carpeta está vacía y tendrás que copiarlas tú). A continuación, genera un diccionario cuyos pares clave,valor estén formados por la intensidad y el número de píxeles con dicha intensidad, respectivamente. A continuación, una vez generada la codificación Huffman, calcular el número medio de bits por píxel que obtendríamos con esta codificación sobre la imagen. Compara la compresión alcanzada con cada una de las imágenes. ¿Serías capaz de encontrar imágenes de las que hemos trabajado en prácticas con la que obtendríamos una mayor compresión?

In [10]:
def analiza_huffman(ruta_imagen):
    img = cv2.imread(ruta_imagen,0)
    if img is None:
        return
    claves,valor = np.unique(img, return_counts=True)
    freqtable = dict(zip(claves,valor))
    huffman_codes = huffman(freqtable)

    pixeles_total = img.shape[0] * img.shape[1]
    bits_total = 0
    
    for intensidad, codigo in huffman_codes.items():
        frecuencia = freqtable[intensidad]
        codigo_long = len(codigo)
        bits_total += (frecuencia*codigo_long)

    promedio_bits = bits_total / pixeles_total
    return promedio_bits

print(analiza_huffman('images/35.png'))


6.643714904785156


### Ejercicio 2: Truncado de bloques

Vamos a implementar el algoritmo de truncado de bloques para comprimir 
imágenes. Aunque  realmente no vamos a realizar la codificación, este programa nos va a servir para ver una simulación  de cómo quedaría la imagen si aplicásemos esta técnica.

Para ello, crea un programa que reciba una imagen de entrada y un tamaño de bloque n. En primer lugar, vamos a transformar la imagen de manera que el número de filas/columnas que tenga sea múltiplo de n. Una vez transformada, vamos a pasar a tratar cada una de los bloques de manera individual.

Para cada bloque, calcular la media aritmética de los valores y la desviación típica. Una vez calculados, calcula los valores val1 y val2. Recuerda que ahora deberíamos transformar la intensidad de cada píxel en un nuevo valor:
$$val_1=\mu-\sigma\text{ si } x \le \mu$$
$$val_2=\mu+\sigma\text{ si } x > \mu$$
¿Eres capaz de observar diferencias significativas cuando n=2 o 4? ¿Qué error se obtiene entre la imagen original y la imagen "comprimida"? ¿Qué ocurre conforme aumentamos el valor de n? 

In [ ]:
def recorta_imagen(imagen, n):
    """
    Esta función recibe una imagen y un tamaño n y recorta la imagen de tal manera que el número final de filas/columnas 
    sea múltiplo de n. La funció debe recortar las filas de la derecha/abajo.
    """
    limite_y, limite_x = (imagen.shape[0]//n)*n, (imagen.shape[1]//n)*n
    imagen_recortada = imagen[:limite_y,:limite_x]
            
    return imagen_recortada
    

In [ ]:
def transforma_bloque(bloque):
    """
    Esta función recibe un bloque de píxeles y calcula su media y desviación. A continuación sustituye cada píxel por 
    val1 y val2 en función de si es menor o no que la media. La función deveulve dicho valor
    """
    bloque = bloque.astype(np.float32)
    bloque_final = np.zeros_like(bloque)
    media = np.mean(bloque,axis=(2,3))
    media = media.reshape(media.shape[0],media.shape[1],1,1)
    sigma = np.std(bloque,axis=(2,3))
    sigma = sigma.reshape(sigma.shape[0], sigma.shape[1], 1, 1)
    bloque_final = np.where(bloque >media, media+sigma, media-sigma)
    return bloque_final.astype(np.uint8)
    

In [11]:
def truncado_bloques(imagen, n):
    imagen_recortada = recorta_imagen(imagen, n).astype(np.float32)
    imagen_final = np.zeros_like(imagen_recortada)
    F,C = imagen_recortada.shape[0]//n, imagen_recortada[1]//n

    reshaped = imagen_recortada.reshape(F,n,C,n)
    bloques = reshaped.transpose(0,2,1,3)

    imagen_final = transforma_bloque(bloques)
    imagen_final = imagen_final.transpose(0,2,1,3)
    imagen_final = imagen_final.reshape((imagen_recortada.shape[0],imagen_recortada.shape[1]))

    return imagen_final.astype(np.uint8)


### Transformada de Haar

Implementar la codificación de una imagen basada en la transformada de Haar. Para ello, escribe  un    programa    que    reciba    una    imagen    y    un    ratio    de    compresión.    La    función  H = CBHAAR(tamaño_matriz)  nos  devuelve  la  matriz  de  cambio  de  base  de  Haar  del  tamaño  que queramos.  En  nuestro  caso,  el  tamaño  de  la  matriz  (asumimos  imágenes  cuadradas  de dimensión potencia de 2). El programa aplicar la operación Y = HXH’. A partir de la imagen transformada,  cambiar  a  0  el  número  de  valores  indicado  por  el  ratio  de  entrada.  Volver  al espacio inicial y mostrar la nueva imagen.

Analiza  qué  error  (medido como media  de  las  diferencias  al  cuadrado)  existe  entre  la  imagen 
original y la imagen comprimida con ratio 2, 3,
etc.

In [ ]:
def CBHAAR(WidthOfSquareMatrix):
    n = WidthOfSquareMatrix
    level = np.log2(n)
    if 2**level < n:
        print ("please ensure the value of input parameter is the power of 2")
    else:
        H = np.array([1])
        NC = 1./np.sqrt(2)
        NC = 1
        LP = np.array([1,1])
        HP = np.array([1, -1])
        for i in range(int(level)):
            H = NC * np.vstack((np.kron(H, LP), np.kron(np.eye(H.shape[0]), HP)))
        H = np.linalg.inv(H)
        return H.T

In [ ]:
def recorta_imagen_2(imagen):
    """
    Esta función recibe una imagen y la recorta de tal manera que el número final de filas/columnas 
    sea potencia de 2. La funció debe recortar las filas de la derecha/abajo.
    """
    imagen = imagen[:2**int(np.log2(imagen.shape[0])),:2**int(np.log2(imagen.shape[0]))]
    return imagen

In [12]:
def pon_ceros(imagen, ratio):
    """
    Esta función recibe una imagen transformada y un ratio. En primer lugar, la función debe contar cuántos 
    elementos de imagen son ceros (puedes contabilizar como ceros aquellos que sean menores de 10**-8 en valor absoluto). 
    Una vez contados, puedes calcular fácilmente cuántos elementos distintos de cero hay. Supongamos que existen x píxeles 
    distintos de cero. Con la variable ratio, puedes calcular el número de elementos distintos de cero (y) que deben quedar después de la 
    compresión, de forma que x/y=ratio. Pon a cero aquellos elementos de imagen de menor valor absoluto hasta que queden 
    únicamente y píxeles distintos de cero.
    """
    img = imagen.copy()
    cero_count = np.sum(np.abs(img) < 10e-8)
    no_cero_count = img.size - cero_count

    no_cero_after = int(no_cero_count / ratio)
    eliminate_count = no_cero_count - no_cero_after

    img = img.ravel()
    indices = np.argsort(np.abs(img))

    img[indices[cero_count:cero_count+eliminate_count]] = 0
    img = img.reshape(imagen.shape)
    
    return img

In [ ]:
def compresion_haar(imagen, ratio):
    #Recortar la imagen
    # Obtener la matriz de cambio de base
    # Obtener la matriz transformada Y = HXH'
    # LLama a pon ceros para truncar la imagen y obtener Y2
    # Obtener la matriz inversa X2
    X = recorta_imagen_2(imagen.astype(np.float32))
    H = CBHAAR(X.shape[0])
    Y = H @ X @ H.T
    Y2 = pon_ceros(Y,ratio)
    X2 = np.linalg.inv(H) @ Y2 @ np.linalg.inv(H.T)
    return X2

Una vez analizadas las imágenes comprimidas con la transformada, prueba a repetir el proceso pero esta vez dividiendo la imagen en bloques pequeños, 8x8 por ejemplo (siempre potencia de 2), y generando la matriz de cambio de base de dicho tamaño. Con el mismo ratio, analiza si se obtienen mejores compresiones en términos de calidad de la imagen final.

In [ ]:
def compresion_haar_bloques(imagen, ratio, n):
    img = imagen.copy()
    for i in range(img.shape[0]//n):
        img = recorta_imagen_2(imagen)
        H = CBHAAR(img.shape[0])
        Y = H @ H.T
        Y2 = pon_ceros(Y, ratio)
        X2 = np.linalg.pinv(Y2)
        
    return X2
    